## Education and Relative Wealth Index Map of Madagascar

In [25]:
%pip install pyproj geopandas shapely keplergl

Note: you may need to restart the kernel to use updated packages.


In [26]:
import pandas as pd
import geopandas as gpd
from pyproj import CRS
from keplergl import KeplerGl
from shapely.geometry import Point
from shapely.geometry import Polygon
import json

### Data preprocessing

In [27]:
### rwi = relative wealth index
rwi_df = pd.read_csv("mdg_relative_wealth_index.csv")

### The 3rd administrative boundary of Madagascar is the municipalities (communes)
adm_gdf = gpd.read_file("mdg_admbnda_adm3_BNGRC_OCHA/mdg_admbnda_adm3_BNGRC_OCHA_20181031.shp")

### All education places (schools, etc) from Open Street Map
edu_places_gdf = gpd.read_file("hotosm_mdg_education_facilities_points_shp/hotosm_mdg_education_facilities_points.shp")

In [28]:
rwi_df.shape, adm_gdf.shape, edu_places_gdf.shape

((43639, 4), (1579, 17), (1681, 10))

In [29]:
### Convert rwi_df into GeoDataFrame
geometry = [Point(lon, lat) for lon, lat in zip(rwi_df['longitude'], rwi_df['latitude'])]
rwi_gdf  = gpd.GeoDataFrame(rwi_df , geometry=geometry)

### Project the CRS code of communes_gdf to rwi_gdf
rwi_gdf.crs = CRS('EPSG:4326')
rwi_gdf  = rwi_gdf.to_crs(adm_gdf.crs)

#### Join the administration boundaries to the RWI & Education datasets

In [30]:
### Save the communes_gdf geometry then join it with rwi_gdf
adm_gdf['boundary_geometry'] = adm_gdf.geometry
rwi_with_adm_gdf = gpd.sjoin(rwi_gdf, adm_gdf, how='left', predicate='within')

In [31]:
### Join with edu_places_gdf
edu_places_with_adm_gdf = gpd.sjoin(edu_places_gdf, adm_gdf, how='left', predicate='within')

In [32]:
rwi_with_adm_gdf.isna().sum()

latitude                 0
longitude                0
rwi                      0
error                    0
geometry                 0
index_right            407
ADM0_PCODE             407
ADM0_EN                407
ADM1_PCODE             407
ADM1_EN                407
ADM1_TYPE              407
ADM2_PCODE             407
ADM2_EN                407
ADM2_TYPE              407
ADM3_PCODE             407
ADM3_EN                407
ADM3_TYPE              407
PROV_CODE_             407
OLD_PROVIN             407
PROV_TYPE              407
NOTES                43622
SOURCE                 407
boundary_geometry      407
dtype: int64

407 Points (of 43639) are found outside of Madagascar boundary => Acceptable

In [52]:
rwi_with_adm_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 43639 entries, 0 to 43638
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   latitude           43639 non-null  float64 
 1   longitude          43639 non-null  float64 
 2   rwi                43639 non-null  float64 
 3   error              43639 non-null  float64 
 4   geometry           43639 non-null  geometry
 5   index_right        43232 non-null  float64 
 6   ADM0_PCODE         43232 non-null  object  
 7   ADM0_EN            43232 non-null  object  
 8   ADM1_PCODE         43232 non-null  object  
 9   ADM1_EN            43232 non-null  object  
 10  ADM1_TYPE          43232 non-null  object  
 11  ADM2_PCODE         43232 non-null  object  
 12  ADM2_EN            43232 non-null  object  
 13  ADM2_TYPE          43232 non-null  object  
 14  ADM3_PCODE         43232 non-null  object  
 15  ADM3_EN            43232 non-null  object  
 16  A

In [33]:
### Group by communes and get the metrics (mean rwi & std rwi)
grouped_adm_with_rwi_gdf = rwi_with_adm_gdf.groupby('ADM3_PCODE').agg({
    'rwi':['mean','std']
})
grouped_adm_with_rwi_gdf.columns = ['_'.join(col).strip() for col in grouped_adm_with_rwi_gdf.columns.values]
grouped_adm_with_rwi_gdf.head()

,rwi_mean,rwi_std
ADM3_PCODE,,
MG11101001,1.505000,NaN
MG11101002,1.681333,0.256929
MG11101003,1.767000,0.312541
MG11101004,1.485000,0.320907
MG11101005,1.661400,0.300056


In [51]:
grouped_adm_with_rwi_gdf.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1568 entries, MG11101001 to MG72716350
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   rwi_mean  1568 non-null   float64
 1   rwi_std   1546 non-null   float64
dtypes: float64(2)
memory usage: 36.8+ KB


In [34]:
edu_places_with_adm_gdf.operatorty.unique()

array([None, 'public', 'private', 'community', 'religious', 'consortium',
       'catholic'], dtype=object)

In [35]:
edu_places_with_adm_gdf.isna().sum()

osm_id                  0
operatorty           1348
source               1660
name                  169
amenity                 0
addrfull             1665
building             1678
addrcity             1639
capacitype           1681
geometry                0
index_right             0
ADM0_PCODE              0
ADM0_EN                 0
ADM1_PCODE              0
ADM1_EN                 0
ADM1_TYPE               0
ADM2_PCODE              0
ADM2_EN                 0
ADM2_TYPE               0
ADM3_PCODE              0
ADM3_EN                 0
ADM3_TYPE               0
PROV_CODE_              0
OLD_PROVIN              0
PROV_TYPE               0
NOTES                1502
SOURCE                  0
boundary_geometry       0
dtype: int64

Only 169 of 1681 buildings are not named. The amenity defines the buildings type. The rest of edu_places columns are useless (high missing value) for now.

In [36]:
### Group by communes
grouped_adm_with_edu_places_gdf = edu_places_with_adm_gdf.groupby('ADM3_PCODE').agg({
    'amenity':'count',
})
grouped_adm_with_edu_places_gdf.head()

,amenity
ADM3_PCODE,
MG11101001,47
MG11101002,29
MG11101003,22
MG11101004,24
MG11101005,34


In [37]:
adm_final_gdf = adm_gdf[['ADM0_EN','ADM1_EN','ADM2_EN','ADM3_EN','ADM3_PCODE','boundary_geometry']]
adm_final_gdf.columns = ['Country','Region','District','Commune','ADM3_PCODE','boundary_geometry']

### Merge with metrics
adm_with_rwi_and_edu_places_gdf = adm_final_gdf.merge(grouped_adm_with_rwi_gdf,
                                                on='ADM3_PCODE',
                                                how='left')
adm_with_rwi_and_edu_places_gdf = adm_with_rwi_and_edu_places_gdf.merge(grouped_adm_with_edu_places_gdf,
                                                                        on='ADM3_PCODE',
                                                                        how='left')
adm_with_rwi_and_edu_places_gdf.shape
adm_with_rwi_and_edu_places_gdf.columns = ['Country', 
                                           'Region', 
                                           'District', 
                                           'Commune', 
                                           'ADM3_PCODE',
                                           'geometry', 
                                           'rwi_mean', 
                                           'rwi_std', 
                                           'edu_places_count']

In [38]:
adm_with_rwi_and_edu_places_gdf.head()

,Country,Region,District,Commune,ADM3_PCODE,geometry,rwi_mean,rwi_std,edu_places_count
0,Madagascar,Analamanga,1er Arrondissement,1er Arrondissement,MG11101001,"POLYGON ((47.50556 -18.89146, 47.50563 -18.891...",1.505000,NaN,47.0
1,Madagascar,Analamanga,2e Arrondissement,2e Arrondissement,MG11101002,"POLYGON ((47.55842 -18.91178, 47.55857 -18.911...",1.681333,0.256929,29.0
2,Madagascar,Analamanga,3e Arrondissement,3e Arrondissement,MG11101003,"POLYGON ((47.51365 -18.87834, 47.51775 -18.879...",1.767000,0.312541,22.0
3,Madagascar,Analamanga,4e Arrondissement,4e Arrondissement,MG11101004,"POLYGON ((47.50262 -18.91043, 47.50261 -18.910...",1.485000,0.320907,24.0
4,Madagascar,Analamanga,5e Arrondissement,5e Arrondissement,MG11101005,"POLYGON ((47.53500 -18.85464, 47.53518 -18.854...",1.661400,0.300056,34.0


In [39]:
### Keep only the necessary columns from the administrative boundaries
edu_places_with_adm_final_gdf = edu_places_with_adm_gdf[['name',
                                                         'amenity',
                                                         'operatorty',
                                                         'ADM0_EN',
                                                         'ADM1_EN',
                                                         'ADM2_EN',
                                                         'ADM3_EN',
                                                         'geometry']]

edu_places_with_adm_final_gdf.columns = ['Place name',
                                         'Amenity',
                                         'Category',
                                         'Country',
                                         'Region',
                                         'District',
                                         'Commune',
                                         'geometry']

### KeplerGL Map

Kepler doesn't support NaN type so we fill the missing rwi_mean by the min(all_rwi_mean) and rwi_std by 0 and the others by "missing".
Why use the minimum of the relative wealth index (rwi) kpi? because we suppose that where there is no rwi there is no population or not enough population data to be estimated. (low population density = low rwi)

In [40]:
adm_with_rwi_and_edu_places_gdf.isna().sum()

Country                0
Region                 0
District               0
Commune                0
ADM3_PCODE             0
geometry               0
rwi_mean              11
rwi_std               33
edu_places_count    1125
dtype: int64

In [41]:
edu_places_with_adm_final_gdf.isna().sum()

Place name     169
Amenity          0
Category      1348
Country          0
Region           0
District         0
Commune          0
geometry         0
dtype: int64

In [42]:
### we fill the missing rwi_mean by the minimum rwi_mean and rwi_std by 0
adm_with_rwi_and_edu_places_gdf.rwi_mean.fillna(adm_with_rwi_and_edu_places_gdf.rwi_mean.min(),inplace=True)
adm_with_rwi_and_edu_places_gdf.rwi_std.fillna(0,inplace=True)

### fill missing edu_places_count by 0
adm_with_rwi_and_edu_places_gdf.edu_places_count.fillna(0,inplace=True)

### fill categorical features by "missing"
edu_places_with_adm_final_gdf['Place name'].fillna('missing',inplace=True)
edu_places_with_adm_final_gdf['Category'].fillna('missing',inplace=True)

C:\Users\tokya\AppData\Local\Temp\ipykernel_29468\1265230388.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  edu_places_with_adm_final_gdf['Place name'].fillna('missing',inplace=True)
C:\Users\tokya\AppData\Local\Temp\ipykernel_29468\1265230388.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  edu_places_with_adm_final_gdf['Category'].fillna('missing',inplace=True)


In [43]:
### Load pre-registrated configuration
#config_dict = json.load(open('config.json'))

In [44]:
### Convert the Dataframe we used for metrics calculation into GeoDataframe
adm_with_rwi_and_edu_places_gdf = gpd.GeoDataFrame(adm_with_rwi_and_edu_places_gdf, geometry="geometry")
edu_places_with_adm_final_gdf = gpd.GeoDataFrame(edu_places_with_adm_final_gdf, geometry="geometry")

In [54]:
map_1 = KeplerGl(height=1000, data={"Metrics per Commune": adm_with_rwi_and_edu_places_gdf,
                                   "Education places":edu_places_with_adm_final_gdf
                                   })  #, config=config_dict)

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


In [55]:
### Configure the KeplerGL Map before saving it
map_1

KeplerGl(data={'Metrics per Commune':          Country      Region            District             Commune   
…

In [47]:
### Get and save configuration
config_dict = map_1.config
config_dict
with open('config_education_and_rwi.json', 'w') as fp:
    json.dump(config_dict, fp)

In [48]:
### Save Map
map_1.save_to_html(data={"Metrics per Commune": adm_with_rwi_and_edu_places_gdf,
                         "Education places":edu_places_with_adm_final_gdf
                         }, file_name='Education and Relative Wealth of Madagascar.html')


Map saved to Education and Relative Wealth of Madagascar.html!
